In [1]:
%config Completer.use_jedi = False

In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
import torchvision.transforms as transforms
import torch.nn.functional as F
from torch import optim

import matplotlib.pyplot as plt
import numpy as np
import plotly.express as px
from tqdm.notebook import tqdm

In [3]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = datasets.CIFAR10(
    root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(
    trainset, batch_size=32, shuffle=True, num_workers=2)

testset = datasets.CIFAR10(
    root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(
    testset, batch_size=32, shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
px.imshow(torch.movedim(trainset[0][0], 0, 2))

In [6]:
classes[trainset[0][1]]

'frog'

In [44]:
def test(net, dataloader, device='cuda:0'):
    net = net.to(device)
    conf_matrix = np.zeros((10, 10))
    for i, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        with torch.no_grad():
            pred = net(X)
            pred_cls = pred.argmax(1)
            for j in range(len(pred_cls)):
                conf_matrix[y[j]][pred_cls[j]] += 1
    return conf_matrix

In [48]:
def train(net, dataloader, optimizer, loss_func, epochs=100, device='cuda:0'):
    n = len(trainloader.dataset)
    net.to(device)
    train_acc = np.zeros(epochs)
    test_acc = np.zeros(epochs)
    train_loss = np.zeros(epochs)
    test_loss = np.zeros(epochs)
    
    for epoch in tqdm(range(epochs)):
        epoch_ok = 0
        for i, (X, y) in enumerate(dataloader):
            X, y = X.to(device), y.to(device)
            optimizer.zero_grad()
            pred = net(X)
            loss = loss_func(pred, y)
            
            train_loss[epoch] += loss.item()
            pred_cls = pred.argmax(1)
            epoch_ok += sum(pred_cls == y)
            
            loss.backward()
            optimizer.step()
        
        conf_matrix = test(net, testloader)    
        test_acc[epoch] = np.diag(conf_matrix).sum() / np.sum(conf_matrix)
        train_acc[epoch] = epoch_ok / n
        
        print(f'Epoch {epoch}: train loss: {train_loss[epoch]:.4f}, train acc: {train_acc[epoch]:.3f}, '
              f'test acc: {test_acc[epoch]}')

In [33]:
class Net(nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=(3,3), padding=1)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=(3,3), padding=1)
        self.conv3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(3,3), padding=1)
        self.pool = nn.MaxPool2d(kernel_size=(2,2))
        self.gap = nn.MaxPool2d(kernel_size=(8,8))
        
        self.linear = nn.Sequential(
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 10),
            nn.ReLU()
        )
        
    
    def forward(self, x):
        # conv block
        x = self.conv1(x)
        x = self.pool(F.relu(x))
        x = self.conv2(x)
        x = self.pool(F.relu(x))
        x = self.conv3(x)
        x = self.gap(F.relu(x))
        x = x.squeeze()
        # mlp block
        x = self.linear(x)
        return x
    

In [86]:
class Net(nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=(3,3), padding=1)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=(3,3), padding=1)
        self.conv3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(3,3), padding=1)
        self.conv4 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=(3,3), padding=1)
        self.pool = nn.MaxPool2d(kernel_size=(2,2))
        self.gap = nn.MaxPool2d(kernel_size=(16, 16))
        self.bn1 = nn.BatchNorm2d(16)
        self.bn2 = nn.BatchNorm2d(32)
        self.bn3 = nn.BatchNorm2d(64)
        self.bn4 = nn.BatchNorm2d(128)
        
        self.linear = nn.Sequential(
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(32, 10),
            nn.ReLU()
        )
        
    
    def forward(self, x):
        # conv block
        x = self.conv1(x)
        x = self.bn1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = self.bn2(x)
        x = F.relu(x)
        x = self.conv3(x)
        x = self.bn3(x)
        x = F.relu(x)
        x = self.pool(x)
        x = self.conv4(x)
        x = self.bn4(x)
        x = F.relu(x)
        x = self.gap(x)

        x = x.squeeze()
        # mlp block
        x = self.linear(x)
        return x
    

In [87]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [88]:
net = Net().to(device)

In [91]:
loss_func = nn.CrossEntropyLoss()
adam = optim.SGD(net.parameters(), lr=0.01, momentum=0.9)

In [ ]:
train(net, trainloader, adam, loss_func, 30)

  0%|          | 0/30 [00:00<?, ?it/s]

Epoch 0: train loss: 2895.6760, train acc: 0.267, test acc: 0.2804
Epoch 1: train loss: 2791.8459, train acc: 0.276, test acc: 0.2853
Epoch 2: train loss: 2722.9210, train acc: 0.287, test acc: 0.2755
Epoch 3: train loss: 2712.7681, train acc: 0.291, test acc: 0.2837
Epoch 4: train loss: 2632.5181, train acc: 0.314, test acc: 0.3174
Epoch 5: train loss: 2595.5600, train acc: 0.327, test acc: 0.2915
Epoch 6: train loss: 2515.4739, train acc: 0.351, test acc: 0.3673
Epoch 7: train loss: 2451.6955, train acc: 0.374, test acc: 0.4042
Epoch 8: train loss: 2341.0621, train acc: 0.414, test acc: 0.4214
Epoch 9: train loss: 2210.2629, train acc: 0.456, test acc: 0.4768
Epoch 10: train loss: 2083.5239, train acc: 0.489, test acc: 0.5017
Epoch 11: train loss: 1976.1048, train acc: 0.523, test acc: 0.5207
Epoch 12: train loss: 1902.4885, train acc: 0.540, test acc: 0.5248
Epoch 13: train loss: 1844.2700, train acc: 0.557, test acc: 0.5271
Epoch 14: train loss: 1810.1991, train acc: 0.569, test ac

- lr scheduler,
- 